In [2]:
import databaseOps, fields
import numpy as np
import json

In [3]:
conn = databaseOps.connectToDb()
cur = conn.cursor()

In [14]:
def score(aRows, bRows):
    numMatches, editDist, naiveDist = compare(aRows, bRows)
    score = numMatches - editDist - naiveDist
    return score/len(aRows)

# rows need to be ordered by row num
def compare(aRow, bRow):
    numMatches = 0
    editDists, naiveDists = [], []
    numFields = min(len(aRow), len(bRow))
    if numFields == 0:
        return 0
    for j in range(numFields):
        numMatches += sum([1 for i in range(numFields) if aRow[i] == bRow[i]])
        editDists.append(sum([editDistance(str(aRow[i]), str(bRow[i])) for i in range(numFields)]))
        naiveDists.append(sum([naiveEdit(str(aRow[i]), str(bRow[i])) for i in range(numFields)]))
#     print(numMatches, sum(editDists), sum(naiveDists))
    return (numMatches - sum(editDists) - sum(naiveDists))/numFields #more negative = more unlike
    return numMatches - sum(editDists) - sum(naiveDists) #more negative = more unlike

# Shortest edit distance (Levenstein)
def editDistance(aField, bField):
    if aField == bField:
        return 0
    n, m = len(aField), len(bField)
    if aField == "None":
        return m/m
    if bField == "None":
        return n/n
    dp = [[0 for x in range(n+1)] for x in range(m+1)] 
    for i in range(m+1): 
        for j in range(n+1):
            if i == 0: 
            	dp[i][j] = j
            elif j == 0: 
                dp[i][j] = i
            elif aField[j-1] == bField[i-1]: 
                dp[i][j] = dp[i-1][j-1] 
            else: 
                dp[i][j] = 1 + min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1])    
  
    return dp[m][n]/(n+m) #normalized is better

# delete from end of bField until match beginning of aField, then add rest of aField
def naiveEdit(aField, bField):
    if aField == bField:
        return 0
    n, m = len(aField), len(bField)
    if aField == "None":
        return m/m
    if bField == "None":
        return n/n
    if (len(aField) > len(bField)):
        a, b = aField, bField
    else:
        b, a = aField, bField
    numEdits = 0
    while aField.find(bField) != 0 and len(bField) > 0:
        bField = bField[:-1]
        numEdits += 1
    numEdits += len(aField) - len(bField)
    return numEdits/(n+m)

In [15]:
### compare just head

fourA = [
'887546/000095011717000076/0000950117-17-000076', 
'893739/000121102217000006/0001211022-17-000006', 
'1305168/000089924317006824/0000899243-17-006824', 
'1653649/000120919117010174/0001209191-17-010174', 
'893739/000129672017000004/0001296720-17-000004', 
'1173479/000117347917000003/0001173479-17-000003']

aTo4 = dict()
for idx in range(len(fourA)):
    aHead = cur.execute("select * from form4Head where accNum = '" + fourA[idx]+ "';").fetchall()

    query = "select B.accNum as bAcc from(select * from form4head group by accNum) A, (select * from form4head group by accNum) B where A.accNum='"+ fourA[idx] + "' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate;" 

    matches = cur.execute(query).fetchall() #grouped by [(aAcc, bAcc)]
    probDist = dict()
    headScores, totalScores = dict(), dict()
    for match in matches:
        bAcc = match[0]
        bHead = cur.execute("select * from form4Head where accNum = '" + bAcc + "';").fetchall()
        
        mHeadScores = np.zeros(len(bHead))
        
        for a in aHead:
            for j in range(len(bHead)):
                b = bHead[j]
                mHeadScores[j] = compare(a, b)

        totalScore = sum(mHeadScores)
        headScores[bAcc], totalScores[bAcc] = mHeadScores, totalScore
        
    scores = totalScores.values()
    minScore = min(scores)
    total = 0
    for key in totalScores.keys():
        if minScore < 0:
            totalScores[key] += abs(minScore) + 1 #prevent from nan
        total += totalScores[key]
    for key in totalScores.keys():
        probDist[key] = totalScores[key]/total
        
    values = sorted(probDist.values(), reverse=True)
    aTo4[fourA[idx]] = dict()
    for key in probDist.keys():
        if probDist[key] in values[:4]:
            aTo4[fourA[idx]][key] = probDist[key]

In [17]:
print(json.dumps(aTo4, indent=2))

{
  "1173479/000117347917000003/0001173479-17-000003": {
    "1267238/0001209191-17-020499": 0.00875276396018988,
    "1267238/0001209191-17-020492": 0.008863037406424716,
    "1208099/0000014272-17-000084": 0.009219876151935038,
    "1173479/0001173479-17-000002": 0.011294647475343816
  },
  "1653649/000120919117010174/0001209191-17-010174": {
    "1423824/0001209191-17-010156": 0.0071604044355710935,
    "880266/0001209191-17-010011": 0.0072057948570116645,
    "1391094/0001225208-17-002902": 0.007102899455874793,
    "1653649/0001209191-17-010146": 0.008204129510762463
  },
  "887546/000095011717000076/0000950117-17-000076": {
    "1620083/0001174947-17-000112": 0.00546146278109076,
    "1365555/0001365555-17-000013": 0.00540239075819866,
    "887546/0000950117-17-000073": 0.006156764121461026,
    "887546/0000950117-17-000072": 0.006433398341640108
  },
  "1305168/000089924317006824/0000899243-17-006824": {
    "1305168/0000899243-17-004719": 0.006771359124288842,
    "1305168/0000

In [13]:
print(json.dumps(aTo4, indent=2))

{
  "1173479/000117347917000003/0001173479-17-000003": {
    "1267238/0001209191-17-020499": 0.0083607426842177,
    "1267238/0001209191-17-020492": 0.00838446017386864,
    "1208099/0000014272-17-000084": 0.008463006341704772,
    "1173479/0001173479-17-000002": 0.009653845051601136
  },
  "1653649/000120919117010174/0001209191-17-010174": {
    "1423824/0001209191-17-010156": 0.0071151076121033306,
    "1438731/0000947871-17-000113": 0.007095841428523645,
    "1391094/0001225208-17-002902": 0.0071373502150283916,
    "1653649/0001209191-17-010146": 0.0076491313575607935
  },
  "887546/000095011717000076/0000950117-17-000076": {
    "887546/0000950117-17-000072": 0.00595811531951541,
    "922864/0001229384-17-000014": 0.005395997793155307,
    "922864/0001229384-17-000012": 0.0053924463634449655,
    "887546/0000950117-17-000073": 0.005807838916083542
  },
  "1305168/000089924317006824/0000899243-17-006824": {
    "1305168/0000899243-17-004719": 0.006369010413742831,
    "1305168/0000

In [15]:
### compare head, dt, and ndt

fourA = [
'887546/000095011717000076/0000950117-17-000076', 
'893739/000121102217000006/0001211022-17-000006', 
'1305168/000089924317006824/0000899243-17-006824', 
'1653649/000120919117010174/0001209191-17-010174', 
'893739/000129672017000004/0001296720-17-000004', 
'1173479/000117347917000003/0001173479-17-000003'] #[list of accNum]
#fourA = ['1305168/000089924317006824/0000899243-17-006824'] #[list of accNum]
#fourA = ['1653649/000120919117010174/0001209191-17-010174'] #need to debug

aTo4 = dict()
for idx in range(len(fourA)):
    aHead = cur.execute("select * from form4Head where accNum = '" + fourA[idx]+ "';").fetchall()
    aDT = cur.execute("select * from form4dT where accNum = '" + fourA[idx]+ "';").fetchall()
    aNDT = cur.execute("select * from form4ndT where accNum = '" + fourA[idx]+ "';").fetchall()

    query = "select B.accNum as bAcc from(select * from form4head group by accNum) A, (select * from form4head group by accNum) B where A.accNum='"+ fourA[idx] + "' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate;" 

    matches = cur.execute(query).fetchall() #grouped by [(aAcc, bAcc)]
    probDist = dict()
    headScores, dtScores, ndtScores, totalScores = dict(), dict(), dict(), dict()
    for match in matches:
        bAcc = match[0]
        bHead = cur.execute("select * from form4Head where accNum = '" + bAcc + "';").fetchall()
        bDT = cur.execute("select * from form4dT where accNum = '" + bAcc + "';").fetchall()
        bNDT = cur.execute("select * from form4ndT where accNum = '" + bAcc + "';").fetchall()
        
        if (len(aDT) != 0 and len(bDT) == 0) or (len(aNDT) != 0 and len(bNDT) == 0):
            continue
        
        scoreLen = max(len(bHead), len(bDT), len(bNDT))
        mHeadScores, mDtScores, mNdtScores = np.zeros(scoreLen), np.zeros(scoreLen), np.zeros(scoreLen)
        
        for a in aHead:
            for j in range(len(bHead)):
                b = bHead[j]
                mHeadScores[j] = compare(a, b)
        
#         for a in aDT:
#             for j in range(len(bDT)): #if dt is empty then score is 0 (maybe ndt update?)
#                 b = bDT[j]
#                 mDtScores[j] = compare(a, b)
#         for a in aNDT:
#             for j in range(len(bNDT)):
#                 b = bNDT[j]
#                 mNdtScores[j] = compare(a, b)

        totalScore = sum([mHeadScores[p] + mDtScores[p] + mNdtScores[p] for p in range(len(mHeadScores))])
        headScores[bAcc], dtScores[bAcc], ndtScores[bAcc], totalScores[bAcc] = mHeadScores, mDtScores, mNdtScores, totalScore
    scores = totalScores.values()
    minScore = min(scores)
    total = 0
    for key in totalScores.keys():
        if minScore < 0:
            totalScores[key] += abs(minScore) + 1 #prevent from nan
        total += totalScores[key]
    for key in totalScores.keys():
        probDist[key] = totalScores[key]/total
        
    values = sorted(probDist.values(), reverse=True)
    aTo4[fourA[idx]] = dict()
    for key in probDist.keys():
        if probDist[key] in values[:4]:
            aTo4[fourA[idx]][key] = probDist[key]

In [14]:
aTo4

{'1173479/000117347917000003/0001173479-17-000003': {'104918/0000104918-17-000060': 0.009402764693499432,
  '104918/0000104918-17-000062': 0.009466962674099699,
  '1053507/0001338678-17-000013': 0.009456511840048492,
  '5981/0000899243-17-007506': 0.009393806835741255},
 '1305168/000089924317006824/0000899243-17-006824': {'1131096/0001131096-17-000020': 0.007997111223308276,
  '1305168/0000899243-17-004716': 0.008944180228529048,
  '1305168/0000899243-17-004721': 0.009160406028807763,
  '1305168/0000899243-17-004732': 0.009137702319778498},
 '1653649/000120919117010174/0001209191-17-010174': {'1219959/0001179110-17-002224': 0.009311824347337504,
  '1467373/0001467373-17-000118': 0.009308262568959369,
  '1637502/0001032220-17-000011': 0.009303365123689433,
  '1653649/0001209191-17-010146': 0.009809582875681913},
 '887546/000095011717000076/0000950117-17-000076': {'1620083/0001174947-17-000112': 0.007130178799164856,
  '732717/0001127602-17-003691': 0.007118711538603304,
  '887546/000095

In [ ]:
# what fields was changed? given 4/a and 4
# diff in head, diff in dt, diff in ndt -> {fieldname: (before/4, after/4a)}

matches = [('887546/000095011717000076/0000950117-17-000076', '887546/000095011717000076/0000950117-17-000076')]

for match in matches:
    fourA, four = match[0], match[1]
    aHead = cur.execute("select * from form4Head where accNum = '" + fourA + "';").fetchall()    
    aDT = cur.execute("select * from form4dT where accNum = '" + fourA+ "';").fetchall()
    aNDT = cur.execute("select * from form4ndT where accNum = '" + fourA+ "';").fetchall()
    bHead = cur.execute("select * from form4Head where accNum = '" + four + "';").fetchall()
    bDT = cur.execute("select * from form4dT where accNum = '" + four + "';").fetchall()
    bNDT = cur.execute("select * from form4ndT where accNum = '" + four + "';").fetchall()
    
    probDist = dict()
    headScores, dtScores, ndtScores, totalScores = dict(), dict(), dict(), dict()
    for match in matches:
        bAcc = match[0]
        
        

In [3]:
import sys
!{sys.executable} -m pip install pandas

    100% |████████████████████████████████| 14.4MB 788kB/s  a 0:00:011
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
